In [197]:
import pandas as pd
from pathlib import Path
from fuzzywuzzy import fuzz
from pydantic import BaseModel
from typing import Literal
from ollama import Client
import json

In [198]:
# MODEL = "llama3.2"
# MODEL = "granite3-dense:8b"
MODEL = "qwen2.5-coder:14b"
RESULT_THRESHOLD = 60

In [199]:
class MantisResponse(BaseModel):
  solution: str
  reason: str

In [200]:
root = Path().resolve()
data_path = root / "data"
csv_file = data_path / "mantis_export.csv"

In [201]:
csv_df = pd.read_csv(csv_file)

In [202]:
def apply_ratio(row, search):
    return fuzz.ratio(row, search)
def apply_partial_ratio(row, search):
    return fuzz.partial_ratio(row, search)
def apply_token_sort_ratio(row, search):
    return fuzz.token_sort_ratio(row, search)
def apply_token_set_ratio(row, search):
    return fuzz.token_set_ratio(row, search)
def apply_weighted_ratio(row, search):
    return fuzz.WRatio(row, search)

filt = csv_df['Description'].notna()
csv_df = csv_df[filt]

search = "How to fix picture in picture?"
csv_df['ratio_%'] = csv_df['Summary'].apply(lambda row:apply_ratio(row, search))
# csv_df['partial_ratio_%'] = csv_df['Summary'].apply(lambda row:apply_partial_ratio(row, search))
# csv_df['token_sort_ratio_%'] = csv_df['Summary'].apply(lambda row:apply_token_sort_ratio(row, search))
# csv_df['token_set_ratio_%'] = csv_df['Summary'].apply(lambda row:apply_token_set_ratio(row, search))
# csv_df['weighted_ratio_%'] = csv_df['Description'].apply(lambda row:apply_weighted_ratio(row, search))

In [203]:
csv_df.sort_values(by='ratio_%', inplace=True, ascending=False)
csv_df = csv_df.head(5)
csv_df

,Id,Project,Reporter,Assigned To,Priority,Severity,Category,Date Submitted,View Status,Updated,Summary,Status,Target Date,Description,Notes,ratio_%
1119,23540,Limerick Full Scope Glass Panel Simulator - CO...,smcdonald,mmansour,high,minor,(No Category),01/10/24,public,05/24/24,10C653 - Implement DEHC Ovation Picture in Pic...,Closed,NaN,Graphics are from Ovation Classroom Suite,@estone 05/24/2024\n653 DEHC ovation implement...,52
9993,14473,Salem Glass Panel Simulator,wmoran,wmoran,normal,major,(No Category),06/27/18,public,12/17/18,Power Factor Meter not depicted,Closed,NaN,The Power Factor Meter ( Panel Item CB21 ) on ...,@ylacombe 09/16/2018\nThe updated drawing was ...,52
2368,22917,Grand Gulf Simulator Modernization ENT22215,rgoldman,jstone,normal,minor,(No Category),08/17/23,public,08/23/23,Micro Mod pictures,Closed,NaN,See attached.\r\n,@rgoldman 08/17/2023\nMove pics. I also have ...,50
13349,11084,TMI THOR Upgrade - EXE14051,jtesmer,jtesmer,normal,minor,(No Category),09/16/16,public,12/15/16,Tune THUGAP to achieve center line temperature,Closed,NaN,THTFUAV appears to be a bit too hot. THUGAP ap...,@jtesmer 12/15/2016\nSpoke with nuclear engine...,50
16450,6496,Browns Ferry Upgrade,achampion,dwebb,normal,major,Core/RCS Model Upgrade,10/26/13,public,09/22/14,How to monitor Keff,Closed,NaN,What is parameter for monitoring critical cond...,@dwebb 11/07/2013\nCRXKEFF is the Keff at the ...,49


In [204]:
def generate_response(search, fuzzy_match_list):
    client = Client()
    messages = [{'role': 'user',
                 'content': f'For user questions: {search}, search these bug reports that were ranked by match precentage: {fuzzy_match_list}. The match percentage \
for each bug report is enclosed by brackets, Like the following: [[match %]]. weigh the match % extremely heavily when picking a category.\
Remember, variables in the summary that end in parenthese such as thk_flownw(408) are not generic and most likely do not match the numbering scheme the \
user is asking about.'}]
    
    json_str = client.chat(model=MODEL, 
               messages=messages,
            #    options={'num_ctx': 16384, 'temperature': 0},
               options={'temperature': 0},
               format=MantisResponse.model_json_schema())['message']['content']
    response = json.loads(json_str)
    print('*' * 200)
    print(f'Response for "{search}": [{response['solution']}]\n')
    print(f'Reason for response: {response['reason']}')
    print(f'\nMATCH LIST:\n{fuzzy_match_list}')
    print('*' * 200)


row_dict = csv_df.to_dict(orient='records') 
results = ""
result_found = False
results_list = list()
for row in row_dict:
    if int(row['ratio_%']) > RESULT_THRESHOLD:
        results += f'{row['Summary']}:\n \
{row['Description']}:\n \
{row['Notes']}: [[{row['ratio_%']}%]].\n \
end of DR {row['Id']}\n'
        result_found = True
        results_list.append(f'DR {row["Id"]}: {row['Summary']} ({row['Project']})')
        
if not result_found:
    results = f'No results over [[{RESULT_THRESHOLD}%]].'
else:
    generate_response(search, results)

for r in results_list:
    print(r)

********************************************************************************************************************************************************************************************************
Response for "How to fix picture in picture?": [To fix picture-in-picture issues, please refer to the following bug reports ranked by match percentage:]

Reason for response: The user is seeking a solution for fixing picture-in-picture functionality. By providing relevant bug reports sorted by their match percentages, we can offer the most pertinent information to address the issue.

MATCH LIST:

********************************************************************************************************************************************************************************************************
